In [1]:
import pandas as pd
import re

In [2]:
import sys
sys.path.append('../')

In [3]:
from metaexplainercode import metaexplainer_utils
from metaexplainercode import codeconstants

In [4]:
synthesis_rag_eval_file =codeconstants.SYNTHESIS_FOLDER + 'Evaluations_rag.csv'
synthesis_rag_eval_pd = pd.read_csv(synthesis_rag_eval_file)


In [8]:
synthesis_rag_eval_pd.groupby('explanation_type').mean()

,Unnamed: 0,__index_level_0__,faithfulness,answer_relevancy,context_utilization
explanation_type,,,,,
Case Based Explanation,1.558824,0.617647,0.210808,0.680842,0.635857
Contrastive Explanation,2.180000,0.580000,0.353184,0.576914,0.623678
Counterfactual Explanation,2.863636,1.590909,0.173888,0.600016,0.449911
Data Explanation,1.921053,0.500000,0.171326,0.811240,0.891315
Rationale Explanation,1.672414,0.775862,0.351634,0.645636,0.770173


In [4]:
user_responses = codeconstants.SYNTHESIS_FOLDER + '/eval_files/user_responses/'

In [5]:
all_subfolders = metaexplainer_utils.get_subfolders_in_folder(user_responses)

In [6]:
system_responses_list = []
per_explanation_response_list = []
user_responses_list = []

per_evaluation_pattern = 'evaluation_set_\d+.xlsx'

for response_folder in all_subfolders:
    system_responses_list.append(pd.read_excel(response_folder + '/OverallSystemEvaluationSet.xlsx'))
    user_responses_list.append(pd.read_excel(response_folder + '/UserFamiliarityQuestionnaire.xlsx'))

    all_files = metaexplainer_utils.get_files_in_folder(response_folder)

    for eval_file in all_files:
        eval_file = eval_file.split('/')[-1]
        matched_eval = re.match(per_evaluation_pattern, eval_file)

        if matched_eval is not None:
            matched_eval_file = matched_eval[0]
            #print('Match', matched_eval_file)
            per_explanation_response_list.append(pd.read_excel(response_folder + '/' + matched_eval_file, index_col=[0], engine="openpyxl"))
            break

In [80]:
per_explanation_response_df = pd.concat(per_explanation_response_list, ignore_index=True)
per_explanation_response_df.head(5)

,Question,Predicted Explanation Type,Feature groups in Question,Explanation of Matched Subset,Explanation of Explainer Outputs,Metrics,This explanation is useful to my goals.,This explanation has sufficient detail.,The explanation is sufficiently complete.,I understand this AI system correctly due to the explanation.,The explanation is sufficiently complete.
0,Why did the model determine that a 40-year-old...,Rationale Explanation,"[{'Age': '40', 'Sex': 'Female', 'BMI': '30', '...","1).Based on the query results, there are 10 en...",1).The decision-making process in this machine...,Metric Value\n0 Average rule...,1.0,2.0,2.0,1.0,NaN
1,How have different gender groups been compared...,Rationale Explanation,"[{'Sex': '', 'Diabetes': ''}]",1).There are no matches in the dataset based o...,1).The rules derived for classification in the...,Metric Value\n0 Average rule...,0.0,0.0,0.0,0.0,NaN
2,How do the predictions for Diabetes change for...,Counterfactual Explanation,"[{'BMI': '30 and age in [40', '60]?': ''}]","1).Based on the query results, there are two m...","1).Based on the data provided, the counterfact...",Metric Value\n0 ...,0.0,0.0,0.0,0.0,NaN
3,"If the patient was male instead of female, wou...",Counterfactual Explanation,"[{'Male': '', 'Sex': ''}, {'Female': '', 'Sex'...","1).Based on the query results, there are no fu...",1).The counterfactual explanation based on the...,Metric Value\n0 ...,1.0,1.0,2.0,2.0,NaN
4,Why include the Diabetes Pedigree Function in ...,Contrastive Explanation,"[{'DiabetesPedigreeFunction': '', 'Diabetes': ...","1).Based on the query results, there are no ma...",1).The feature importance value for 'Age' in t...,Metric Value\n0 Faithfulness 0.65...,1.0,1.0,2.0,1.0,NaN


In [81]:
#dropping duplicate column

per_explanation_response_df.rename({ele:'The explanation is sufficiently complete.' for ele in list(per_explanation_response_df.columns) if 'The explanation is sufficiently complete.' in ele},axis=1,inplace=True)
per_explanation_response_df = per_explanation_response_df.groupby(per_explanation_response_df.columns, axis=1).agg(np.max)

per_explanation_response_df = per_explanation_response_df.replace(1,3) #yes with 3
per_explanation_response_df = per_explanation_response_df.replace(2, 1) #maybe with 1
per_explanation_response_df = per_explanation_response_df.replace(3, 2) #yes with 2 

In [71]:
per_explanation_response_df.columns

Index(['Explanation of Explainer Outputs', 'Explanation of Matched Subset',
       'Feature groups in Question',
       'I understand this AI system correctly due to the explanation.',
       'Metrics', 'Predicted Explanation Type', 'Question',
       'The explanation is sufficiently complete.',
       'This explanation has sufficient detail.',
       'This explanation is useful to my goals.'],
      dtype='object')

In [82]:
per_explanation_response_df.groupby('Predicted Explanation Type').mean()

,I understand this AI system correctly due to the explanation.,The explanation is sufficiently complete.,This explanation has sufficient detail.,This explanation is useful to my goals.
Predicted Explanation Type,,,,
Case Based Explanation,0.285714,0.214286,0.642857,0.214286
Contrastive Explanation,1.000000,0.710526,1.026316,1.026316
Counterfactual Explanation,0.785714,0.571429,0.833333,0.714286
Data Explanation,1.300000,1.080000,1.340000,1.460000
Rationale Explanation,0.698413,0.444444,0.666667,0.761905


In [83]:
per_explanation_response_df = per_explanation_response_df.replace(0,1) #yes with 3
per_explanation_response_df = per_explanation_response_df.replace(1, 3) #maybe with 1
per_explanation_response_df = per_explanation_response_df.replace(2, 5) #yes with 2 

In [84]:
per_explanation_response_df.groupby('Predicted Explanation Type').mean()

,I understand this AI system correctly due to the explanation.,The explanation is sufficiently complete.,This explanation has sufficient detail.,This explanation is useful to my goals.
Predicted Explanation Type,,,,
Case Based Explanation,3.000000,3.142857,3.428571,3.142857
Contrastive Explanation,3.789474,3.526316,3.789474,3.894737
Counterfactual Explanation,3.619048,3.428571,3.571429,3.476190
Data Explanation,4.080000,3.880000,4.080000,4.280000
Rationale Explanation,3.476190,3.317460,3.539683,3.603175


In [88]:
per_explanation_response_df.to_excel(codeconstants.SYNTHESIS_FOLDER + '/eval_files/filled_per_explanation_responses.xlsx')

In [85]:
#which dimensions are from trust, curiosity and satisfication

system_responses_df = pd.concat(system_responses_list)

system_responses_df.groupby(['Evaluation Question']).mean()

,Likert Rating
Evaluation Question,
I am confident in the MetaExplainer. I feel that it works well. [Error frequency],2.285714
I like using the system for decision making. [Satisfication],2.214286
I want to know more about what the AI did.,4.214286
I want to know what the AI would have done if something had been different.,4.214286
I want to know why the AI did not make some other decision.,4.071429
I want to understand what the AI will do next.,3.928571
I was surprised by the AI’s actions and want to know what I missed.,3.071429
The MetaExplainer can perform the task better than a novice human user.,2.714286
The outputs of the MetaExplainer are predictable. [Memorability],3.428571


In [89]:
system_responses_df.to_excel(codeconstants.SYNTHESIS_FOLDER + '/eval_files/filled_system_evaluation_responses.xlsx')

In [91]:
system_responses_df = pd.read_excel(codeconstants.SYNTHESIS_FOLDER + '/eval_files/filled_system_evaluation_responses.xlsx')
system_responses_df.head(10)

,Unnamed: 0,Evaluation Question,Likert Rating
0,0,I am confident in the MetaExplainer. I feel th...,3.0
1,1,The outputs of the MetaExplainer are predictab...,4.0
2,2,The MetaExplainer can perform the task better ...,3.0
3,3,I like using the system for decision making. [...,4.0
4,4,I want to know more about what the AI did.,4.0
5,5,I want to understand what the AI will do next.,4.0
6,6,I want to know why the AI did not make some ot...,2.0
7,7,I want to know what the AI would have done if ...,2.0
8,8,I was surprised by the AI’s actions and want t...,3.0
9,0,I am confident in the MetaExplainer. I feel th...,5.0


In [87]:
import numpy as np

user_details_df = pd.concat(user_responses_list)
user_details_df.head(20)

,Question,Answer,Unnamed: 2
0,How familiar are you with Explainable AI? (Lik...,5,NaN
1,What expalinable AI tools / methods have you u...,"LIME, SHAP, TDA, Feature Attribution",NaN
2,How familiar are you with Diabetes? (Likert ra...,2,NaN
3,Were the questions too specific to Diabetes or...,1,NaN
0,How familiar are you with Explainable AI? (Lik...,4,NaN
1,What expalinable AI tools / methods have you u...,Counterfactual Explainability,NaN
2,How familiar are you with Diabetes? (Likert ra...,4,NaN
3,Were the questions too specific to Diabetes or...,2,NaN
0,How familiar are you with Explainable AI? (Lik...,3,NaN
1,What expalinable AI tools / methods have you u...,Never used any myself,NaN


In [90]:
user_details_df.to_excel(codeconstants.SYNTHESIS_FOLDER + '/eval_files/filled_user_questionnaire.xlsx')